In [68]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('market_data.log'),
        logging.StreamHandler()
    ]
)

In [69]:
import json
import os
import subprocess
import sys
import datetime
from typing import Any, Dict, List, Optional, Type, Union
import tinkoff.invest
from tinkoff.invest import PortfolioRequest, PortfolioPosition, Client, RequestError, CandleInterval, HistoricCandle, \
    OrderType, OrderDirection, Quotation, InstrumentIdType, InstrumentStatus
from tinkoff.invest.services import InstrumentsService
from datetime import datetime
import pandas as pd
import os
import sys
import numpy as np
import importlib
from concurrent.futures import ThreadPoolExecutor

import sys
sys.path.append('/Users/aeshef/Documents/GitHub/kursach')
import run_with_config
importlib.reload(run_with_config)
from run_with_config import run_with_config


In [70]:
#####

import market_data
importlib.reload(market_data)
from market_data import run_pipeline_market

######

import fundamental_data
importlib.reload(fundamental_data)
from fundamental_data import run_pipeline_fundamental

######

import tech_analysis
importlib.reload(tech_analysis)
from tech_analysis import run_pipeline_technical

######

import news_pipeline
importlib.reload(news_pipeline)
from news_pipeline import NewsPipeline

######

import data_integration
importlib.reload(data_integration)
from data_integration import run_pipeline_integration

######

import bonds_processor
importlib.reload(bonds_processor)
from bonds_processor import run_pipeline_bonds

## Globals

In [65]:
token = "t.GVEetX09IVL9qECDUhjThfpg5TrE2Oxw9cUnAIOxcHh35M2sv1YMy9LBfkt73F3WlV_Ik08etZOVrUNI8StLBQ"
kursach_directory_path = "/Users/aeshef/Documents/GitHub/kursach"

tickers = pd.read_csv(f"{kursach_directory_path}/data/tickers.csv")
tickers_list = [
    'SBER',  # Сбербанк
    'GAZP',  # Газпром
    'LKOH',  # Лукойл
    'GMKN',  # ГМК "Норильский никель"
    'ROSN',  # Роснефть
    'TATN',  # Татнефть
    'MTSS',  # МТС
    'ALRS',  # АК Алроса
    'SNGS',  # Сургутнефтегаз
    'NVTK',  # Новатэк
    'MVID',  # М.Видео
    'PHOR',  # ФосАгро
    'SIBN',  # Сибнефть
    'AFKS',  # АФК Система
    'MAGN',  # Магнитогорский металлургический комбинат
    'RUAL'   # Русал
]

start_date="2024-01-01"
end_date="2025-04-15"
timeframe="1h"

YOUR_API_ID = 28994010
YOUR_API_HASH = "6e7a57fdfe1a10b0a3434104b42badf2"

overall_df = f'{kursach_directory_path}/data/df.csv'

## Initial parser

In [58]:
run_pipeline_market(
    tickers=tickers_list,
    start_date=start_date,
    end_date=end_date,
    token=token
)

2025-04-23 02:45:45,167 - tinkoff.invest.logging - INFO - f7d232585910a7958af6702fe027e361 Shares


Congratulations! BBG004731489 is a correct figi for GMKNCongratulations! BBG004730N88 is a correct figi for SBER

Congratulations! BBG004731032 is a correct figi for LKOH
Congratulations! BBG004731354 is a correct figi for ROSN
Congratulations! BBG004730RP0 is a correct figi for GAZP

LKOH data statistics:
  - Total rows: 378321
  - First date: 2024-01-02 07:16:00
  - Last date: 2025-04-14 20:49:00
Congratulations! BBG004RVFFC0 is a correct figi for TATN

SBER data statistics:
  - Total rows: 412112
  - First date: 2024-01-02 07:16:00
  - Last date: 2025-04-14 20:49:00

SBER data:
  - Timeframe: 1min
  - Total rows: 412112
  - First date: 2024-01-02 07:16:00
  - Last date: 2025-04-14 20:49:00
Congratulations! BBG004S681W1 is a correct figi for MTSS

ROSN data statistics:
  - Total rows: 417181
  - First date: 2024-01-02 07:16:00
  - Last date: 2025-04-14 20:49:00
Congratulations! BBG004S68B31 is a correct figi for ALRS

GMKN data statistics:
  - Total rows: 343876
  - First date: 2024-

{'SBER':                       date    open   close     min     max  volume
 0      2024-01-02 07:16:00  268.87  271.04  271.04  268.87      17
 1      2024-01-02 07:17:00  268.87  271.00  271.07  268.87     194
 2      2024-01-02 07:18:00  269.00  270.83  271.00  269.00      94
 3      2024-01-02 07:19:00  269.91  269.91  270.83  269.91      66
 4      2024-01-02 07:20:00  269.91  270.72  270.76  269.91     102
 ...                    ...     ...     ...     ...     ...     ...
 412107 2025-04-14 20:45:00  296.45  296.36  296.46  296.36    1464
 412108 2025-04-14 20:46:00  296.36  296.49  296.49  296.36    1395
 412109 2025-04-14 20:47:00  296.49  296.48  296.50  296.48    1327
 412110 2025-04-14 20:48:00  296.49  296.55  296.55  296.48    1020
 412111 2025-04-14 20:49:00  296.54  296.50  296.55  295.83    8854
 
 [412112 rows x 6 columns],
 'GAZP':                       date    open   close     min     max  volume
 0      2024-01-02 07:16:00  160.34  159.40  160.34  159.40      22
 1

## Fundamental

In [59]:
run_pipeline_fundamental(ticker_list=tickers_list)


Обработка тикеров: ['SBER', 'GAZP', 'LKOH', 'GMKN', 'ROSN', 'TATN', 'MTSS', 'ALRS', 'SNGS', 'NVTK', 'MVID', 'PHOR', 'SIBN', 'AFKS', 'MAGN', 'RUAL']

[DEBUG] Список показателей (value_float не NaN) для каждого тикера и года:
  SBER, 2023 -> 55 показателей: ['Чистый операц доход, млрд руб', 'Чист. проц. доходы, млрд руб', 'Чист. комисс. доход, млрд руб', 'Доходы по цб, млрд руб', 'Число клиентов, млн', 'Чистая прибыль, млрд руб', 'Див.выплата, млрд руб', 'Дивиденд, руб/акцию', 'Див доход, ао, %', 'Дивиденд ап, руб/акцию', 'Див доход, ап, %', 'Дивиденды/прибыль, %', 'Опер. расходы, млрд руб', 'Расх на персонал, млрд руб', 'Процентные расходы, млрд руб', 'Создание резервов, млрд руб', 'Наличность, млрд руб', 'Чистый долг, млрд руб', 'Активы банка, млрд руб', 'Капитал, млрд руб', 'Кредитный портфель, млрд руб', 'Кредиты юрлицам, млрд руб', 'Кредиты физлицам, млрд руб', 'Депозиты, млрд руб', 'Депозиты юрлиц, млрд руб', 'Депозиты физлиц, млрд руб', 'Дост.осн капитала, %', 'Дост. общ капитала

## Tech

In [ ]:
# внимание, я чуть поменял механизм - смотри, теперь для каждого года у меня общие фундпоказатели хранятся в таком виде /Users/aeshef/Documents/GitHub/kursach/data/processed_data/{ticker}/fundamental_analysis/{year}/common.csv. но тут есть нюанс - не во всех из них данные - числа. так получилось, что у некоторых компаний были пустые отчета за 2024 год, но из-за этогоо глупо терять информацию, что у большинства компаний есть много общих показателей, по которым их можно ранжироваить и сравнивать, как это делаешь ты. из-за этого мне пришлось прописать механизм - если у большинства компаний за год есть этот общий показатель - парами его, но если у какой-то компании его не было (так как не было отчета) - мы просто проставляем специальную строку внутри csv. вот примеры: 1) /Users/aeshef/Documents/GitHub/kursach/data/processed_data/ALRS/fundamental_analysis/2024/common.csv выглядит так - Показатель,value_float
# "BV/акцию, руб",49.7
# "CAPEX, млрд руб",58.9
# "CAPEX/Выручка, %",25.0
# "EBITDA, млрд руб",78.6
# "EPS, руб",2.87
# "EV, млрд руб",531.7
# EV/EBITDA,6.76
# "FCF, млрд руб",-17.6
# "FCF/акцию, руб",-2.39
# P/BV,1.16
# P/E,20.0
# P/FCF,-24.1
# P/S,1.77
# "R&D/CAPEX, %",0.0
# "ROA, %",3.2
# "ROE, %",5.7
# "Активы, млрд руб",658.7
# "Амортизация, млрд руб",33.3
# "Баланс стоимость, млрд руб",366.3
# "Выручка, млрд руб",239.1
# "Див доход, ао, %",0.0
# "Дивиденды/прибыль, %",0.0
# "Долг, млрд руб",196.6
# Долг/EBITDA,1.37
# "Доходность FCF, %",-4.2
# "Капитализация, млрд руб",423.8
# "Наличность, млрд руб",88.7
# "Опер. расходы, млрд руб",54.8
# "Опер.денежный поток, млрд руб",41.3
# "Операционная прибыль, млрд руб",30.5
# "Рентаб EBITDA, %",32.9
# "Цена акции ао, руб",57.5
# "Число акций ао, млн",7365.0
# "Чистая прибыль, млрд руб",21.2
# "Чистая рентаб, %",8.8
# "Чистые активы, млрд руб",371.0
# "Чистый долг, млрд руб",107.9
# тогда как 2) /Users/aeshef/Documents/GitHub/kursach/data/processed_data/GAZP/fundamental_analysis/2024/common.csv выглядит так Показатель,value_float
# "EPS, руб",NO_DATA
# "Чистая прибыль, млрд руб",NO_DATA
# "Опер. расходы, млрд руб",NO_DATA
# "EV, млрд руб",NO_DATA
# "Число акций ао, млн",NO_DATA
# P/E,NO_DATA
# "Дивиденды/прибыль, %",NO_DATA
# "Чистый долг, млрд руб",NO_DATA
# "Капитализация, млрд руб",NO_DATA
# "ROA, %",NO_DATA
# "Цена акции ао, руб",NO_DATA
# "Див доход, ао, %",NO_DATA
# "ROE, %",NO_DATA
# "Наличность, млрд руб",NO_DATA
# "Операционная прибыль, млрд руб",NO_DATA
# "CAPEX/Выручка, %",NO_DATA
# EV/EBITDA,NO_DATA
# Долг/EBITDA,NO_DATA
# "EBITDA, млрд руб",NO_DATA
# "Выручка, млрд руб",NO_DATA
# "FCF, млрд руб",NO_DATA
# "Опер.денежный поток, млрд руб",NO_DATA
# "Доходность FCF, %",NO_DATA
# "FCF/акцию, руб",NO_DATA
# "BV/акцию, руб",NO_DATA
# "Рентаб EBITDA, %",NO_DATA
# P/BV,NO_DATA
# "CAPEX, млрд руб",NO_DATA
# P/S,NO_DATA
# "Долг, млрд руб",NO_DATA
# "R&D/CAPEX, %",NO_DATA
# "Чистые активы, млрд руб",NO_DATA
# P/FCF,NO_DATA
# "Активы, млрд руб",NO_DATA
# "Баланс стоимость, млрд руб",NO_DATA
# "Амортизация, млрд руб",NO_DATA
# "Чистая рентаб, %",NO_DATA
# . я считаю, что компании, на которые нет данных, можно обработать отдельно - в разбивке на квантиль например считать, что они примерно посередине и так далее по аналогии. реализуй это пожалуйста

In [66]:
run_pipeline_technical()

2025-04-23 02:51:53 - TechAnalysisPipeline - INFO - Инициализирован TechAnalysisPipeline.
2025-04-23 02:51:53,150 - TechAnalysisPipeline - INFO - Инициализирован TechAnalysisPipeline.
2025-04-23 02:51:53 - TechAnalysisPipeline - INFO - === НАЧАЛО ТЕХНИЧЕСКОГО АНАЛИЗА ===
2025-04-23 02:51:53,153 - TechAnalysisPipeline - INFO - === НАЧАЛО ТЕХНИЧЕСКОГО АНАЛИЗА ===
2025-04-23 02:51:53 - TechAnalysisPipeline - INFO - Обработка тикера: GAZP
2025-04-23 02:51:53,195 - TechAnalysisPipeline - INFO - Обработка тикера: GAZP
2025-04-23 02:51:53 - TechAnalysisPipeline - INFO - Используем файл: GAZP.parquet
2025-04-23 02:51:53,196 - TechAnalysisPipeline - INFO - Используем файл: GAZP.parquet
2025-04-23 02:51:53,197 - TechPipeline - INFO - Начало загрузки данных.
2025-04-23 02:51:53,236 - TechPipeline - INFO - Начало расчёта технических индикаторов.
2025-04-23 02:51:53 - TechAnalysisPipeline - INFO - Сохранен CSV с индикаторами: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/GAZP/tech_anal

Index(['date', 'open', 'close', 'low', 'high', 'volume'], dtype='object')


2025-04-23 02:51:53 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/GAZP/tech_analysis/GAZP_MACD.png
2025-04-23 02:51:53,622 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/GAZP/tech_analysis/GAZP_MACD.png
2025-04-23 02:51:53 - TechAnalysisPipeline - INFO - Обработка тикера: SBER
2025-04-23 02:51:53,623 - TechAnalysisPipeline - INFO - Обработка тикера: SBER
2025-04-23 02:51:53 - TechAnalysisPipeline - INFO - Используем файл: SBER.parquet
2025-04-23 02:51:53,624 - TechAnalysisPipeline - INFO - Используем файл: SBER.parquet
2025-04-23 02:51:53,624 - TechPipeline - INFO - Начало загрузки данных.
2025-04-23 02:51:53,659 - TechPipeline - INFO - Начало расчёта технических индикаторов.
2025-04-23 02:51:53 - TechAnalysisPipeline - INFO - Сохранен CSV с индикаторами: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/SBER/tech_analysis/SBER_tech_indicators.cs

Index(['date', 'open', 'close', 'low', 'high', 'volume'], dtype='object')


2025-04-23 02:51:54 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/SBER/tech_analysis/SBER_MACD.png
2025-04-23 02:51:54,045 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/SBER/tech_analysis/SBER_MACD.png
2025-04-23 02:51:54 - TechAnalysisPipeline - INFO - Обработка тикера: LKOH
2025-04-23 02:51:54,045 - TechAnalysisPipeline - INFO - Обработка тикера: LKOH
2025-04-23 02:51:54 - TechAnalysisPipeline - INFO - Используем файл: LKOH.parquet
2025-04-23 02:51:54,046 - TechAnalysisPipeline - INFO - Используем файл: LKOH.parquet
2025-04-23 02:51:54,047 - TechPipeline - INFO - Начало загрузки данных.
2025-04-23 02:51:54,078 - TechPipeline - INFO - Начало расчёта технических индикаторов.
2025-04-23 02:51:54 - TechAnalysisPipeline - INFO - Сохранен CSV с индикаторами: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/LKOH/tech_analysis/LKOH_tech_indicators.cs

...

2025-04-23 02:52:00 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/AFKS/tech_analysis/AFKS_MACD.png
2025-04-23 02:52:00,073 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/AFKS/tech_analysis/AFKS_MACD.png
2025-04-23 02:52:00 - TechAnalysisPipeline - INFO - Обработка тикера: MAGN
2025-04-23 02:52:00,075 - TechAnalysisPipeline - INFO - Обработка тикера: MAGN
2025-04-23 02:52:00 - TechAnalysisPipeline - INFO - Используем файл: MAGN.parquet
2025-04-23 02:52:00,076 - TechAnalysisPipeline - INFO - Используем файл: MAGN.parquet
2025-04-23 02:52:00,078 - TechPipeline - INFO - Начало загрузки данных.
2025-04-23 02:52:00,116 - TechPipeline - INFO - Начало расчёта технических индикаторов.
2025-04-23 02:52:00 - TechAnalysisPipeline - INFO - Сохранен CSV с индикаторами: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/MAGN/tech_analysis/MAGN_tech_indicators.cs

Index(['date', 'open', 'close', 'low', 'high', 'volume'], dtype='object')


2025-04-23 02:52:00 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/MAGN/tech_analysis/MAGN_MACD.png
2025-04-23 02:52:00,593 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/MAGN/tech_analysis/MAGN_MACD.png
2025-04-23 02:52:00 - TechAnalysisPipeline - INFO - Обработка тикера: RUAL
2025-04-23 02:52:00,594 - TechAnalysisPipeline - INFO - Обработка тикера: RUAL
2025-04-23 02:52:00 - TechAnalysisPipeline - INFO - Используем файл: RUAL.parquet
2025-04-23 02:52:00,595 - TechAnalysisPipeline - INFO - Используем файл: RUAL.parquet
2025-04-23 02:52:00,596 - TechPipeline - INFO - Начало загрузки данных.
2025-04-23 02:52:00,629 - TechPipeline - INFO - Начало расчёта технических индикаторов.
2025-04-23 02:52:00 - TechAnalysisPipeline - INFO - Сохранен CSV с индикаторами: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/RUAL/tech_analysis/RUAL_tech_indicators.cs

Index(['date', 'open', 'close', 'low', 'high', 'volume'], dtype='object')


2025-04-23 02:52:01 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/RUAL/tech_analysis/RUAL_MACD.png
2025-04-23 02:52:01,113 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/RUAL/tech_analysis/RUAL_MACD.png
2025-04-23 02:52:01 - TechAnalysisPipeline - INFO - Сводный отчет сохранен в /Users/aeshef/Documents/GitHub/kursach/data/processed_data/tech_analysis_summary.txt
2025-04-23 02:52:01,115 - TechAnalysisPipeline - INFO - Сводный отчет сохранен в /Users/aeshef/Documents/GitHub/kursach/data/processed_data/tech_analysis_summary.txt
2025-04-23 02:52:01 - TechAnalysisPipeline - INFO - === ТЕХНИЧЕСКИЙ АНАЛИЗ ЗАВЕРШЕН ===
2025-04-23 02:52:01,116 - TechAnalysisPipeline - INFO - === ТЕХНИЧЕСКИЙ АНАЛИЗ ЗАВЕРШЕН ===


## News

In [71]:
run_with_config(
    script_path=f"{kursach_directory_path}/pys/data_collection/news_pipeline.py",
    class_or_function="NewsPipeline",
    method="run_pipeline",
    base_dir=kursach_directory_path,
    config_filename="news_pipeline_config.json",
    
    tickers=tickers_list,
    collect_telegram=True,
    telegram_api_id=YOUR_API_ID,
    telegram_api_hash=YOUR_API_HASH,
    telegram_channel="cbrstocks",
    telegram_limit=10000,
    start_date=start_date,
    end_date=end_date,
    use_cached_telegram=True
)

Конфигурация сохранена в /Users/aeshef/Documents/GitHub/kursach/news_pipeline_config.json
Запуск команды: /Library/Developer/CommandLineTools/usr/bin/python3 /Users/aeshef/Documents/GitHub/kursach/pys/data_collection/news_pipeline.py --config /Users/aeshef/Documents/GitHub/kursach/news_pipeline_config.json
Запуск pipeline.run_pipeline с параметрами: {'tickers': ['SBER', 'GAZP', 'LKOH', 'GMKN', 'ROSN', 'TATN', 'MTSS', 'ALRS', 'SNGS', 'NVTK', 'MVID', 'PHOR', 'SIBN', 'AFKS', 'MAGN', 'RUAL'], 'collect_telegram': True, 'telegram_api_id': 28994010, 'telegram_api_hash': '6e7a57fdfe1a10b0a3434104b42badf2', 'telegram_channel': 'cbrstocks', 'telegram_limit': 10000, 'start_date': datetime.date(2024, 1, 1), 'end_date': datetime.date(2025, 4, 15), 'use_cached_telegram': True}
2025-04-23 02:56:50 - NewsPipeline - INFO - Запуск пайплайна анализа новостей для 16 тикеров
2025-04-23 02:56:50 - NewsPipeline - INFO - Диапазон дат: 2024-01-01 - 2025-04-15
2025-04-23 02:56:50 - pymorphy2.opencorpora_dict.wr

0

In [72]:
run_pipeline_integration(tickers=tickers_list, 
                         output_path = overall_df)

2025-04-23 03:02:44,872 - DataIntegrator - INFO - Обработка тикера SBER...
2025-04-23 03:02:44,918 - DataIntegrator - INFO - Данные для SBER загружены за период: 2024-01-02 - 2025-04-14
2025-04-23 03:02:44,920 - DataIntegrator - INFO -   - Обнаружено 5400 пропущенных значений для SBER
2025-04-23 03:02:45,011 - DataIntegrator - INFO -   - Пропущенные значения заполнены методом zero
2025-04-23 03:02:45,012 - DataIntegrator - INFO - Обработка тикера GAZP...
2025-04-23 03:02:45,038 - DataIntegrator - INFO - Данные для GAZP загружены за период: 2024-01-02 - 2025-04-14
2025-04-23 03:02:45,039 - DataIntegrator - INFO -   - Обнаружено 5177 пропущенных значений для GAZP
2025-04-23 03:02:45,131 - DataIntegrator - INFO -   - Пропущенные значения заполнены методом zero
2025-04-23 03:02:45,133 - DataIntegrator - INFO - Обработка тикера LKOH...
2025-04-23 03:02:45,156 - DataIntegrator - INFO - Данные для LKOH загружены за период: 2024-01-02 - 2025-04-14
2025-04-23 03:02:45,157 - DataIntegrator - INF

In [74]:
bonds_results=run_pipeline_bonds(
    # Ожидая снижение ставки, увеличиваем вес доходности
    score_weights={
        'yield': 0.7,           # Больше вес на доходность
        'duration': 0.2,        # Меньше вес на дюрацию
        'tax_benefit': 0.05,    # Меньше вес на налоговую льготу
        'price_discount': 0.05  # Меньше вес на скидку
    },
    market_condition='high_rate',
    select_top_n=5,
    date_range=(start_date, end_date)
)

risk_free_rate = bonds_results['risk_free_rate']


2025-04-23 03:03:29,307 - bonds_pipeline - INFO - Запуск пайплайна обработки облигаций в 2025-04-23 03:03:29
2025-04-23 03:03:29,310 - bonds_pipeline - INFO - Начинаем загрузку и обработку данных по облигациям из /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/csv
2025-04-23 03:03:29,313 - bonds_pipeline - INFO - Успешно загружен файл bond_search_2024-03-26.csv с 19 облигациями от 2024-03-26
2025-04-23 03:03:29,315 - bonds_pipeline - INFO - Успешно загружен файл bond_search_2024-04-10.csv с 20 облигациями от 2024-04-10
2025-04-23 03:03:29,318 - bonds_pipeline - INFO - Успешно загружен файл bond_search_2024-05-24.csv с 15 облигациями от 2024-05-24
2025-04-23 03:03:29,320 - bonds_pipeline - INFO - Успешно загружен файл bond_search_2024-08-09.csv с 27 облигациями от 2024-08-09
2025-04-23 03:03:29,323 - bonds_pipeline - INFO - Успешно загружен файл bond_search_2024-08-10.csv с 30 облигациями от 2024-08-10
2025-04-23 03:03:29,327 - bonds_pipeline - INFO - Успешно загружен ф


Топ рекомендуемых облигаций:
    security_code                  full_name  yield  duration_years  \
336  RU000A105MP6  ДОМОДЕДОВО ФьюэлФ 002Р-01  37.19        0.805000   
352  RU000A106HF5     ЭЛЕКТРОРЕШЕНИЯ 001Р-01  32.58        1.277500   
347  RU000A104ZK2          МВ ФИНАНС 001Р-03  29.83        0.421667   
343  RU000A104YT6          ГК Самолет БО-П12  29.78        0.391667   
335  RU000A10ARS4          МОНОПОЛИЯ 001P-03  28.89        0.841667   

     risk_adjusted_yield  bond_score  weight  
336            46.198758    2.237186  0.1526  
352            25.502935    1.442737  0.0962  
347            70.743083    1.190934  0.2914  
343            76.034043    1.183059  0.3137  
335            34.324752    0.868785  0.1460  
Обработано 24 записей облигаций
Текущая безрисковая ставка: 25.49%

Топ рекомендуемых облигаций:
    security_code                  full_name  yield  duration_years  \
336  RU000A105MP6  ДОМОДЕДОВО ФьюэлФ 002Р-01  37.19        0.805000   
352  RU000A106HF5     

<!-- я делаю курсовой проект. я придерживаюсь этого плана:
# ---  
## 1. Слой Сбора и Агрегации Данных

### 1.1. Рыночные данные
- Источники API: Yahoo Finance, Alpha Vantage для акций. Для облигаций/ОФЗ — выгрузки или ручная подготовка, если нет автосбора.
- Загрузка исторических котировок всех бумаг, которые потенциально могут войти в портфель (акции + risk-free/фонды).
- Сохранение единой “сырых” данных (CSV/Parquet).

### 1.2. Фундаментальные данные
- Используются только последние доступные отчёты по каждой бумаге (например, за 2023–24, чаще нет).
- Допускается ручной ввод/подготовка csv коллегой (дефолтно — только 1 snapshot на тикер).
- Промышленный подход: между апдейтами отчётов — “forward fill” фундаментальных коэффициентов; при вычислениях делается анализ только cross-sectional (по состоянию на одну дату между бумагами).
- Стандартизация (z-score) коэффициентов для последующего скоринга.

### 1.3. Новостной/сентимент-поток
- Автоматизация выгрузки новостей (RSS, Twitter, базовые каналы для акций и risk-free).
- Анализ новостного сентимента (проще — ручная классификация “важные макро/кризисы” для risk-free бумаг).
- Сохранение текстовых и сентиментных фич в отдельные поля.

---

## 2. Слой Предобработки и Признаков

### 2.1. Генерация признаков для акций и risk-free
- Чистка рядов; заполнение пропусков.
- Генерация технических индикаторов (SMA, EMA, RSI, MACD, Bollinger Bands, паттерны — минимум сигналы типа "пересечение SMA").
- Для risk-free инструментов — расчет доходностей/дюраций “на сейчас”, если есть хотя бы скриншот доходности.
- Для всех сегментов — доходности, волатильности, ковариации.

### 2.2. Работа с фундаменталом (ограничена snapshot-режимом!)
- Для единственного года: сравнение бумаг между собой — расчёт z-score фундаментальных метрик на дату.
- Проведение статического ранжирования (квантили, top/bottom группы).
- Генерация сводного композитного score для отбора universe.

### 2.3. Интеграция сентимента и фундаментала
- Корректировка статичного funda-score с учётом свежих новостных сигналов (например, большой рынок негатив — понизить вес funda-score).
- Для risk-free — влияние только “макро”-новостей и текущей ставочной среды.

---

## 3. Модуль анализа и сигналов

### 3.1. Генерация торговых сигналов
- Акции: Buy/Hold/Sell по композитному score (технические + функдаментал + новости).
- Для risk-free: всегда Hold/Buy (core allocation), исключения — при негативных макро/ставках.

### 3.2. Кросс-секционный квантильный отбор
- “Shortlist” бумаг для портфеля: top 30–50% по funda-score, приоритизированных техническими индикаторами.
- Для risk-free инструментов можно просто фиксировать веса или задавать диапазон для оптимизации.

---  

## 4. Оптимизация Портфеля

### 4.1. Модель Марковица (Mean-Variance)
- Оптимизация среди shortlist бумаг (только Buy/Hold).
- Явно фиксируем вес на risk-free (“core allocation”, например 25–35%).
- Остальное распределяем по акциям оптимально по риск-доходности (mean-variance).

### 4.2. Black-Litterman (опционально)
- Возможность внести “view” на основе ранга funda-score и новостных сигналов.
- Коррекция “базовых” рыночных весов с помощью своих score.

---

## 5. Интеграция, отчётность и автоматизация

### 5.1. Финальные сигналы и обработка портфеля
- Взвешенная агрегация сигналов (funda-score × техника × новости).
- Единый пайплайн: загрузка — признаки — сигналы — оптимизация — результат.
- Пайплайны собраны как большие скрипты-автоматизаторы для (квази)ежедневного обновления.

### 5.2. Интерфейс пользователя
- Просмотр результатов
по кнопке (CLI/Jupyter/Telegram-бот).
- Возможность быстро изменить параметры universe/core allocation.

---

## 6. Бэктестинг и визуализация

### 6.1. Бэктест
- Проверка стратегии на годичном горизонте (по доступным данным).
- Особое внимание: для фундаментала — тестирование, как бы работали “портфели top vs bottom квантилей по snapshot”.
- Акцент на анализ разницы “просто держать весь рынок” vs. “отбирать по фунду/score”.

### 6.2. Визуализация
- Графики динамики портфеля, групп, эффективная граница.
- Визуализация сравнения групп, влияния сигналов.

---

## 7. Автоматизация и пайплайны

- Все ключевые этапы реализованы как отдельные большие py-скрипты (классы: DataCollector, FeatureEngineer, SignalGenerator, PortfolioOptimizer, Backtester, Visualizer).
- Каждый пайплайн сохраняет промежуточные дампы для отладки.
- Финальный мастер-пайплайн (“одна кнопка”).
- Делегирование ручных элементов коллегой: подготовка funda/csv, ручная разметка событий/новостей, первичный запуск функций.

---

## 8. Особенности и работа с мало меняющимися фундаментальными данными

- Прямо указывается в отчёте: фундаментальные признаки “медленные”, обновляются редко, анализ кросс-секционный: используется только сравнение между бумагами на одну дату, либо квантилизация и ранжирование при каждом отчёте.
- Расширение базы данных даже на 2–3 года (если возможно) — значительно повысит качество анализа и возможности rolling backtest.
- Все краткосрочные сигналы (техникa, новости) агрегируются для “ускорения” pipeline решений между публикациями отчетов.

---

## 9. Как преподнести в работе/отчёте

> Фундаментальные данные используются в cross-sectional анализе и квантилизации портфелей из-за своей низкой частоты обновления и ограниченной истории. Это соответствует практике индустрии для развивающихся рынков/ограниченных источников. При появлении новых отчетов pipeline автоматически обновляет ранжирование бумаг и состава портфеля. Краткосрочные решения строятся преимущественно на технических и новостных сигналах.

---

## Если понадобится — скину шаблоны py-файлов и классов по этому плану.


на данный момент я нахожусь на таком этапе: 
1) структура проекта
/Users/aeshef/Documents/GitHub/kursach
(├── df.csv - обьединенный датасет с 384 фичами для 17 тикеров за 2024-2025 год
├── figi.txt
├── news_analysis_summary.txt
├── processed_data
│   ├── AFKS
│   │   ├── AFKS.parquet
│   │   ├── AFKS_all_news_processed.csv
│   │   ├── AFKS_news_20250422_1601.csv
│   │   ├── news_analysis
│   │   │   ├── AFKS_daily_events.csv
│   │   │   ├── AFKS_daily_sentiment.csv
│   │   │   ├── AFKS_event_distribution.png
│   │   │   ├── AFKS_ml_features.csv
│   │   │   ├── AFKS_news_with_events.csv
│   │   │   ├── AFKS_news_with_sentiment.csv
│   │   │   ├── AFKS_price_vs_sentiment.png
│   │   │   ├── AFKS_sentiment_distribution.png
│   │   │   ├── AFKS_sentiment_dynamics.png
│   │   │   ├── AFKS_sentiment_features.csv
│   │   │   └── AFKS_topics.txt
│   │   └── tech_analysis
│   │       ├── AFKS_MACD.png
│   │       ├── AFKS_price_indicators.png
│   │       └── AFKS_tech_indicators.csv
...
│   ├── VTBR
│   │   ├── VTBR.parquet
│   │   ├── VTBR_all_news_processed.csv
│   │   ├── VTBR_news_20250422_1601.csv
│   │   ├── news_analysis
│   │   │   ├── VTBR_daily_events.csv
│   │   │   ├── VTBR_daily_sentiment.csv
│   │   │   ├── VTBR_event_distribution.png
│   │   │   ├── VTBR_ml_features.csv
│   │   │   ├── VTBR_news_with_events.csv
│   │   │   ├── VTBR_news_with_sentiment.csv
│   │   │   ├── VTBR_price_vs_sentiment.png
│   │   │   ├── VTBR_sentiment_distribution.png
│   │   │   ├── VTBR_sentiment_dynamics.png
│   │   │   ├── VTBR_sentiment_features.csv
│   │   │   └── VTBR_topics.txt
│   │   └── tech_analysis
│   │       ├── VTBR_MACD.png
│   │       ├── VTBR_price_indicators.png
│   │       └── VTBR_tech_indicators.csv
│   ├── bonds_csv
│   │   ├── bond_search_2023-12-01.csv
│   │   ├── bond_search_2024-03-26.csv
│   │   ├── bond_search_2024-04-10.csv
│   │   ├── bond_search_2024-05-24.csv
│   │   ├── bond_search_2024-08-09.csv
│   │   ├── bond_search_2024-08-10.csv
│   │   ├── bond_search_2024-08-12.csv
│   │   ├── bond_search_2024-09-04.csv
│   │   ├── bond_search_2024-09-13.csv
│   │   ├── bond_search_2024-10-30.csv
│   │   ├── bond_search_2024-11-26.csv
│   │   ├── bond_search_2024-12-04.csv
│   │   ├── bond_search_2025-01-28.csv
│   │   ├── bond_search_2025-02-10.csv
│   │   ├── bond_search_2025-02-14.csv
│   │   └── bond_search_2025-04-22.csv
...
├── raw_data
│   ├── AFKS
│   │   ├── BBG004S68614_2024
│   │   │   ├── 53b67587-96eb-4b41-8e0c-d2e3c0bdd234_20240102.csv
│   │   │   ├── 53b67587-96eb-4b41-8e0c-d2e3c0bdd234_20240103.csv
│   │   │   ├── 53b67587-96eb-4b41-8e0c-d2e3c0bdd234_20240104.csv
│   │   │   ├── 53b67587-96eb-4b41-8e0c-d2e3c0bdd234_20240105.csv
...

├── status.json
├── telegram_news
│   ├── AFKS_telegram_news_20250422.csv
│   ├── ALRS_telegram_news_20250422.csv
│   ├── GAZP_telegram_news_20250422.csv
│   ├── GMKN_telegram_news_20250422.csv
│   ├── LKOH_telegram_news_20250422.csv
│   ├── MAGN_telegram_news_20250422.csv
│   ├── MTSS_telegram_news_20250422.csv
│   ├── MVID_telegram_news_20250422.csv
│   ├── NVTK_telegram_news_20250422.csv
│   ├── PHOR_telegram_news_20250422.csv
│   ├── ROSN_telegram_news_20250422.csv
│   ├── RUAL_telegram_news_20250422.csv
│   ├── SBER_telegram_news_20250422.csv
│   ├── SIBN_telegram_news_20250422.csv
│   ├── SNGS_telegram_news_20250422.csv
│   ├── TATN_telegram_news_20250422.csv
│   ├── VTBR_telegram_news_20250422.csv
│   └── telegram_headlines_20250422_160151.csv
└── tickers.csv) - конец папки data
├── ipynbs
│   ├── market_data.log
│   └── overall_data_pipeline.ipynb - тут в виде ноутбука (потому будет py файлом) расписан полный пайплайн сбора тикеров, тех анализа, фундаментального анализа, извлечения фичей из новостей и получение объединенного датасета со всеми фичами для всех тикеров (df.csv)
├── news_pipeline_config.json
├── pys
│   ├── __pycache__
│   │   ├── event_detector.cpython-313.pyc
│   │   ├── news_feature_extractor.cpython-313.pyc
│   │   ├── news_integration.cpython-313.pyc
│   │   ├── news_preprocessor.cpython-313.pyc
│   │   ├── news_visualizer.cpython-313.pyc
│   │   └── sentiment_analysis.cpython-313.pyc
│   ├── data_integration.py
│   ├── event_detector.py
│   ├── fundamental_data.py
│   ├── market_data.py
│   ├── news_feature_extractor.py
│   ├── news_integration.py
│   ├── news_pipeline.py
│   ├── news_preprocessor.py
│   ├── news_visualizer.py
│   ├── portfolio_optimizer.py
│   ├── run_with_config.py
│   ├── sentiment_analysis.py
│   ├── tech_analysis.py
│   └── technical_indicators.py
└── telegram_session.session

2) я еще вообще никак не работал с данными bonds, так что с ними что-то надо сделать. 

у меня есть 7 дней на выполнение всего курсового проекта, так что сейчас от тебя мне нужно много кода. предложи в соответствии с планом, что мне стоит сейчас сделать -->

In [85]:
sys.path.append('/Users/aeshef/Documents/GitHub/kursach/pys/porfolio_optimization')

#####

import signal_generator
importlib.reload(signal_generator)
from signal_generator import run_pipeline_signal_generator

#####

import portfolio_optimizer
importlib.reload(portfolio_optimizer)
from portfolio_optimizer import PortfolioOptimizer

#####

import backtester
importlib.reload(backtester)
from backtester import Backtester

In [78]:
run_pipeline_signal_generator(
    weight_tech=0.5,
    weight_sentiment=0.3,
    weight_fundamental=0.2
)

2025-04-23 03:04:14,715 - signal_generator - INFO - Запуск пайплайна генерации торговых сигналов
2025-04-23 03:04:14,715 - signal_generator - INFO - Запуск пайплайна генерации торговых сигналов
2025-04-23 03:04:14,716 - signal_generator - INFO - Загрузка данных из /Users/aeshef/Documents/GitHub/kursach/data/df.csv
2025-04-23 03:04:14,716 - signal_generator - INFO - Загрузка данных из /Users/aeshef/Documents/GitHub/kursach/data/df.csv
2025-04-23 03:04:14,907 - signal_generator - INFO - Загружено 7471 строк данных
2025-04-23 03:04:14,907 - signal_generator - INFO - Загружено 7471 строк данных
2025-04-23 03:04:14,908 - signal_generator - INFO - Расчет композитного скора
2025-04-23 03:04:14,908 - signal_generator - INFO - Расчет композитного скора
2025-04-23 03:04:14,912 - signal_generator - INFO - Технический скор рассчитан на основе 6 индикаторов
2025-04-23 03:04:14,912 - signal_generator - INFO - Технический скор рассчитан на основе 6 индикаторов
2025-04-23 03:04:14,916 - signal_generat

In [81]:
portfolio = PortfolioOptimizer(
    risk_free_rate=risk_free_rate,
    min_rf_allocation=0.3,
    max_rf_allocation=0.5
).run_pipeline()

2025-04-23 03:07:51,378 - portfolio_optimizer - INFO - Запуск пайплайна оптимизации портфеля
2025-04-23 03:07:51,378 - portfolio_optimizer - INFO - Запуск пайплайна оптимизации портфеля
2025-04-23 03:07:51,381 - portfolio_optimizer - INFO - Загрузка данных из /Users/aeshef/Documents/GitHub/kursach/data/signals.csv
2025-04-23 03:07:51,381 - portfolio_optimizer - INFO - Загрузка данных из /Users/aeshef/Documents/GitHub/kursach/data/signals.csv
2025-04-23 03:07:51,590 - portfolio_optimizer - INFO - Загружено 7471 строк данных
2025-04-23 03:07:51,590 - portfolio_optimizer - INFO - Загружено 7471 строк данных
2025-04-23 03:07:51,591 - portfolio_optimizer - INFO - Подготовка данных доходностей для оптимизации
2025-04-23 03:07:51,591 - portfolio_optimizer - INFO - Подготовка данных доходностей для оптимизации
2025-04-23 03:07:51,598 - portfolio_optimizer - INFO - Отфильтровано 7030 строк по shortlist и сигналам
2025-04-23 03:07:51,598 - portfolio_optimizer - INFO - Отфильтровано 7030 строк по

In [86]:
Backtester(
    portfolio_weights=portfolio['weights'],
    start_date=start_date,
    end_date=end_date
).run_pipeline()

2025-04-23 03:12:27,684 - backtester - INFO - Запуск пайплайна бэктестирования
2025-04-23 03:12:27,684 - backtester - INFO - Запуск пайплайна бэктестирования
2025-04-23 03:12:27,686 - backtester - INFO - Загрузка данных из /Users/aeshef/Documents/GitHub/kursach/data/signals.csv
2025-04-23 03:12:27,686 - backtester - INFO - Загрузка данных из /Users/aeshef/Documents/GitHub/kursach/data/signals.csv
2025-04-23 03:12:27,930 - backtester - INFO - Загружено 7471 строк данных
2025-04-23 03:12:27,930 - backtester - INFO - Загружено 7471 строк данных
2025-04-23 03:12:27,940 - backtester - INFO - Период бэктеста: 2024-01-02 - 2025-04-14
2025-04-23 03:12:27,940 - backtester - INFO - Период бэктеста: 2024-01-02 - 2025-04-14
2025-04-23 03:12:27,940 - backtester - INFO - Расчет доходностей по тикерам
2025-04-23 03:12:27,940 - backtester - INFO - Расчет доходностей по тикерам
2025-04-23 03:12:27,965 - backtester - INFO - Рассчитаны доходности для 16 тикеров
2025-04-23 03:12:27,965 - backtester - INFO

{'metrics': {'cumulative_return': 0.21698700924370495,
  'annual_return': 0.11153514697957867,
  'annual_volatility': 0.048981012538929954,
  'sharpe_ratio': -150.84344871695114,
  'max_drawdown': -0.034191164071518565,
  'win_rate': 0.9465811965811965},
 'returns':                 AFKS          ALRS          GAZP  GMKN  LKOH  MAGN  \
 date                                                                 
 2024-01-02       NaN           NaN           NaN   NaN   NaN   NaN   
 2024-01-03 -0.001068 -5.067677e-18 -1.226496e-18  -0.0  -0.0  -0.0   
 2024-01-04 -0.003160 -5.028749e-20  2.204938e-19   0.0   0.0  -0.0   
 2024-01-05  0.003765 -6.284171e-19 -8.016843e-19   0.0  -0.0   0.0   
 2024-01-06 -0.000184  1.502925e-19  1.041220e-19   0.0  -0.0   0.0   
 ...              ...           ...           ...   ...   ...   ...   
 2025-04-10 -0.000000 -0.000000e+00  0.000000e+00  -0.0   0.0  -0.0   
 2025-04-11  0.000000  0.000000e+00  0.000000e+00  -0.0   0.0   0.0   
 2025-04-12  0.000000  0